In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
s = pd.read_csv('All_Stock.csv')
s['date'] = pd.to_datetime(s['date'])
s['mktcap'] = s['PRC'] * s['SHROUT']

In [84]:
ff = pd.read_csv('F-F_Research_Data_5_Factors_2x3_daily.CSV')
ff['date'] = pd.to_datetime(ff['Unnamed: 0'])
ff = ff.drop(columns= ['Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 10','Unnamed: 0'])

In [4]:
table = pd.crosstab(s['date'], s['PERMNO'], values = s['mktcap'], aggfunc = sum)
companies = {}
for year in table.index:
    companies[year] = table.loc[year, :].sort_values(ascending = False).index[:100]

In [ ]:
years = list(companies.keys())
lesscomp = pd.DataFrame()
for i in tqdm(years):
    x = s[(s['date'] == i) & (s['PERMNO'].isin(list(companies[i])))]
    lesscomp = pd.concat([lesscomp, x])
s = lesscomp.copy()
s.to_csv('filtered_returns.csv', index=False)

In [ ]:
s = pd.read_csv('filtered_returns.csv')

In [27]:
s['spread'] = s['ASKHI'] - s['BIDLO']
s = s[pd.to_numeric(s['RET'], errors = 'coerce').notna()]
s['RET'] = s['RET'].apply(lambda x : float(x))
s = s[pd.to_numeric(s['RETX'], errors = 'coerce').notna()]
s['RETX'] = s['RETX'].apply(lambda x : float(x))
s['divfac'] = s['RET'] - s['RETX']
s['pctchgvol'] = s.groupby('PERMNO')['VOL'].pct_change()
s

,PERMNO,date,TICKER,BIDLO,ASKHI,PRC,VOL,RET,SHROUT,RETX,mktcap,spread,divfac
0,10401,1970-01-02,T,49.12500,49.87500,49.37500,79500.0,0.015424,549264.0,0.015424,2.711991e+07,0.75000,0.0
1,10604,1970-01-02,ARC,86.00000,90.00000,89.75000,27800.0,0.042090,44389.0,0.042090,3.983913e+06,4.00000,0.0
2,10786,1970-01-02,BS,26.87500,27.25000,27.12500,18000.0,0.000000,44022.0,0.000000,1.194097e+06,0.37500,0.0
3,10890,1970-01-02,BGH,157.25000,159.25000,158.75000,13600.0,0.008737,17232.0,0.008737,2.735580e+06,2.00000,0.0
4,11260,1970-01-02,C,34.25000,35.37500,35.25000,31400.0,0.025455,47942.0,0.025455,1.689956e+06,1.12500,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1286811,92602,2020-12-31,PM,81.29000,82.92000,82.79000,2892403.0,0.016078,1557316.0,0.016078,1.289302e+08,1.63000,0.0
1286812,92611,2020-12-31,V,216.20000,219.82001,218.73000,5922170.0,0.001694,1695677.0,0.001694,3.708954e+08,3.62001,0.0
1286813,92655,2020-12-31,UNH,344.04999,351.09000,350.67999,1969227.0,0.016493,948821.0,0.016493,3.327325e+08,7.04001,0.0
1286814,93002,2020-12-31,AVGO,433.04999,438.50000,437.85001,1015614.0,0.006945,407000.0,0.006945,1.782050e+08,5.45001,0.0


## Column names for 'S'

BIDLO = Bid price
ASKHI = Ask price
PRC = Price
VOL = Volume
RET = Returns
SHROUT = Shares outstanding
Spread = BID ASK spread
divfac = amount dividends effected returns

In [86]:
n = pd.DataFrame()
n['dailyreturns'] = s.groupby('date')['RET'].mean()
n['totalcap'] = s.groupby('date')['mktcap'].sum()
n['chngcap'] = n['totalcap'].pct_change()
n['divfac'] = s.groupby('date')['divfac'].mean()
n['spread'] = s.groupby('date')['spread'].mean()
n['shrout'] = s.groupby('date')['SHROUT'].mean()
n['SDT_RET'] = s.groupby('date')['RET'].std()
n = n.reset_index()
n['date'] = pd.to_datetime(n['date'])
n = pd.merge(n, ff, how = 'left', left_on = 'date', right_on = 'date')
stock = pd.read_csv('All_Stock.csv')
stock = stock[stock['TICKER'] == 'SPY'][['date', 'RET']]
stock['date'] = pd.to_datetime(stock['date'])
n = pd.merge(n, stock, how='left', left_on='date', right_on='date')
n = n[n['date'] >= '1978-10-03']
n = n.rename(columns= {'RET':'SPYRET'})
n['TMW_SPYRET'] = n['SPYRET'].shift(-1)
n = n[pd.to_numeric(n['TMW_SPYRET'], errors = 'coerce').notna()]
n['TMW_SPYRET'] = n['TMW_SPYRET'].apply(lambda x : float(x))
n['UP_DWN'] =  1 * (n['TMW_SPYRET'] > 0)
n = n[pd.to_numeric(n['TMW_SPYRET'], errors = 'coerce').notna()]
n['TMW_SPYRET'] = n['TMW_SPYRET'].apply(lambda x : float(x))
n['UP_DWN'] =  1 * (n['TMW_SPYRET'] > 0)
n

,date,dailyreturns,totalcap,chngcap,divfac,spread,shrout,SDT_RET,Mkt-RF,SMB,HML,RMW,CMA,RF,SPYRET,TMW_SPYRET,UP_DWN
2211,1978-10-03,-0.003019,4.421349e+08,-0.004191,0.000370,0.658750,95254.96,0.009717,-0.31,0.01,0.13,-0.35,-0.04,0.031,-0.017964,0.006098,1
2212,1978-10-04,0.004778,4.449798e+08,0.006434,0.000000,0.823750,95384.82,0.012843,0.38,-0.46,0.05,-0.06,0.00,0.031,0.006098,0.024242,1
2213,1978-10-05,0.002003,4.452071e+08,0.000511,0.000000,0.725000,95507.30,0.010809,0.22,0.19,0.08,0.10,-0.15,0.031,0.024242,0.047337,1
2214,1978-10-06,0.001968,4.461662e+08,0.002154,0.000060,0.665000,95507.30,0.009378,0.23,0.16,-0.07,0.09,-0.09,0.031,0.047337,0.107345,1
2215,1978-10-09,0.011510,4.516742e+08,0.012345,0.000000,0.778750,95254.96,0.011347,0.92,-0.37,-0.19,0.29,-0.02,0.031,0.107345,0.020408,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12860,2020-12-23,0.000843,2.395573e+10,-0.001125,0.000000,27.699945,1748866.63,0.015324,0.10,0.83,2.03,0.29,0.43,0.000,0.000899,0.003890,1
12861,2020-12-24,0.001969,2.401165e+10,0.002334,0.000104,44.291331,1748881.13,0.016186,0.21,-0.43,-0.18,0.23,-0.06,0.000,0.003890,0.008591,1
12862,2020-12-28,0.004419,2.428004e+10,0.011177,0.000000,45.152603,1762696.99,0.012615,0.46,-0.67,0.34,1.39,0.46,0.000,0.008591,-0.001908,0
12863,2020-12-29,0.001078,2.427635e+10,-0.000152,0.000000,33.126933,1768217.95,0.011280,-0.40,-1.42,0.23,0.75,-0.30,0.000,-0.001908,0.001427,1


# Columns for 'n' and comentary on importance

### this table is the aggragate for the top 100 firms in the market for each day

dailyreturns = average daily returns
    Previous days returns could be a large indicator in how the market moves
totalcap = total market capitalization for the 100 firms
    A changee in the total cap of the country could indicate volitility
changecap = percent change in totalcap
divfac = sum divfac for that day
    More dividends one day could put confidence pushing the returns of SPY up
spread = average spread
    Higher spread could indicate less trading and more conservatice ways to come
shrout = shares outstanding in the top 100 firms
    This could indicate macro changes from reports and splits to effect psycology
Mkt-Rf = market returns - risk free
    total market returns not that would be outside of our firms investigates
SMB = smallcap minus largecap
    If small caps react different than large then there could be macro effects
HML = value minus growth companies
    Same as SMB
RMW = robust minus weak
    Similar to SMB
CMA = conservative minus agressive
    Similar to CMA
RF = risk free
    Incase needed, but changing treasury rates could effect the economic outlook
SPYRET = SPY returns for the day
TMW_SPYRET = the SPY returns for the next day
UP_DWN = 1 -> Market up tomorrow, 0 -> Market down tomorrow
    This is the target variable
STD_RET = standard deviation of daily returns
    more volitiltiy could effect the next day of trading.


In [87]:
n.to_csv('daily_aggragate_data.csv', index=False)